https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [72]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 0
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime

from helpers.feature_engineering import dateStrToDayYear, getMappingOfSimilarCategoricalColumns, \
    compareSimilarCategoricalColumns 
from helpers.my_one_hot_encoder import MyOneHotEncoder

In [4]:
rng = np.random.RandomState(seed=random_state)

# Flying to New York City - Target Classes Preprocessing

In [5]:
path_data = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/binary_train_data.csv'

In [6]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(186308, 40)

### Feature Date

In [7]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,8.0,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,43.0,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False


The year attribute is currently the same year, so we could just remove it.
While the flight date attribute brings little information extra information. It could be more useful to convert it to the day of the year since some days could have some particular on a yearly basis.
Although we understand that this attribute will have a correlation with the Quarter, Month, and Day of Month attributes.

In [8]:
df['FL_DATE'].values.shape#[np.newaxis].shape

(186308,)

In [9]:
df['YDAY'] = [dateStrToDayYear(dateStr) for dateStr in df['FL_DATE']]

In [10]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353


In [11]:
df_1 = df.drop(labels=['YEAR', 'FL_DATE'], axis=1)
df_1.shape

(186308, 39)

In [12]:
df_1[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,3,7,3,7,HA,19690,HA,50,12173,HNL,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2,4,2,6,B6,20409,B6,1304,14843,SJU,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,1,3,26,6,B6,20409,B6,164,14986,SRQ,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,1,1,13,3,B6,20409,B6,429,12478,JFK,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,4,12,18,7,DL,19790,DL,786,12953,LGA,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353


### Feature Unique Carrier

In [13]:
carriers = np.unique(df_1['UNIQUE_CARRIER'])
carriers

array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX',
       'WN'], dtype=object)

In [14]:
len(carriers)

12

We see that this feature is categorical with a finite number of carriers which they could be represented with multiple features using one-hot-encoding

In [15]:
unique_carrier_one_hot = MyOneHotEncoder().encodePandasColumn(data_frame = df_1, col_name = 'UNIQUE_CARRIER')

In [16]:
df_2 = pd.concat((df_1, unique_carrier_one_hot), axis=1).drop(labels=['UNIQUE_CARRIER'], axis=1)
df_2.shape

(186308, 49)

In [17]:
df_2[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,...,UNIQUE_CARRIER_1,UNIQUE_CARRIER_2,UNIQUE_CARRIER_3,UNIQUE_CARRIER_4,UNIQUE_CARRIER_5,UNIQUE_CARRIER_6,UNIQUE_CARRIER_7,UNIQUE_CARRIER_8,UNIQUE_CARRIER_9,UNIQUE_CARRIER_10
0,3,7,3,7,19690,HA,50,12173,HNL,"Honolulu, HI",...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,4,2,6,20409,B6,1304,14843,SJU,"San Juan, PR",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26,6,20409,B6,164,14986,SRQ,"Sarasota/Bradenton, FL",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13,3,20409,B6,429,12478,JFK,"New York, NY",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,18,7,19790,DL,786,12953,LGA,"New York, NY",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Also from previous analysis we have established that carrier is redundant. Let's confirm it here and remove this column

In [18]:
assert set(df['UNIQUE_CARRIER']) == set(df['CARRIER'])

In [19]:
df_3 = df_2.drop(labels=['CARRIER'], axis=1)
df_3.shape

(186308, 48)

In [20]:
df_3[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,...,UNIQUE_CARRIER_1,UNIQUE_CARRIER_2,UNIQUE_CARRIER_3,UNIQUE_CARRIER_4,UNIQUE_CARRIER_5,UNIQUE_CARRIER_6,UNIQUE_CARRIER_7,UNIQUE_CARRIER_8,UNIQUE_CARRIER_9,UNIQUE_CARRIER_10
0,3,7,3,7,19690,50,12173,HNL,"Honolulu, HI",HI,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,4,2,6,20409,1304,14843,SJU,"San Juan, PR",PR,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26,6,20409,164,14986,SRQ,"Sarasota/Bradenton, FL",FL,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13,3,20409,429,12478,JFK,"New York, NY",NY,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,18,7,19790,786,12953,LGA,"New York, NY",NY,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### AIRLINE_ID is the SAME AS CARRIER

In [24]:
assert len(np.unique(df_2['AIRLINE_ID'])) == len(np.unique(df_2['CARRIER']))

In [39]:
#airlineId_to_carrier = dict(zip(np.unique(df_2['AIRLINE_ID']), np.unique(df_2['CARRIER'])))
airlineId_to_carrier = {19393: 'WN',
 19690: 'HA',
 19790: 'DL',
 19805: 'AA',
 19930: 'AS',
 19977: 'UA',
 20304: 'OO',
 20366: 'EV',
 20409: 'B6',
 20416: 'NK',
 20436: 'F9',
 21171: 'VX'}

In [40]:
carriers = df_2['CARRIER']
carriers.shape

(186308,)

In [41]:
carriers.values

array(['HA', 'B6', 'B6', ..., 'UA', 'WN', 'AA'], dtype=object)

In [42]:
airlineids = np.array([airlineId_to_carrier[aid] for aid in df_2['AIRLINE_ID']])
airlineids.shape

(186308,)

In [44]:
assert np.all(airlineids == carriers)

With the two asserts above we conclude that the airline_id carries the same information as the carrier which we have already transformed in one hot encoding and thus it must be removed

In [45]:
df_3.shape

(186308, 48)

In [47]:
df_4 = df_3.drop(labels=['AIRLINE_ID'], axis=1)
df_4.shape

(186308, 47)

### Uncertain about Flight number, probably does not bring any extra value (???)

In [50]:
len(np.unique(df_4['FL_NUM']))

4146

In [51]:
np.unique(df_4['FL_NUM'])

array([   1,    2,    3, ..., 7436, 7437, 7439])

### Origin airport id and airport carry the same information

In [57]:
np.unique(df_4['ORIGIN'])

array(['ABQ', 'ACK', 'ALB', 'ATL', 'AUS', 'BGM', 'BGR', 'BHM', 'BNA',
       'BOS', 'BQN', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CHO',
       'CHS', 'CLE', 'CLT', 'CMH', 'CVG', 'DAB', 'DAL', 'DCA', 'DEN',
       'DFW', 'DTW', 'EGE', 'ELM', 'EWR', 'FLL', 'GRR', 'GSO', 'GSP',
       'HNL', 'HOU', 'HPN', 'HYA', 'IAD', 'IAG', 'IAH', 'ILM', 'IND',
       'ISP', 'ITH', 'JAC', 'JAX', 'JFK', 'LAS', 'LAX', 'LEX', 'LGA',
       'LGB', 'MCI', 'MCO', 'MDW', 'MEM', 'MHT', 'MIA', 'MKE', 'MSN',
       'MSP', 'MSY', 'MTJ', 'MVY', 'MYR', 'OAK', 'OMA', 'ORD', 'ORF',
       'PBG', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PSE', 'PSP', 'PWM',
       'RDU', 'RIC', 'RNO', 'ROC', 'RSW', 'SAN', 'SAT', 'SAV', 'SDF',
       'SEA', 'SFO', 'SJC', 'SJU', 'SLC', 'SMF', 'SRQ', 'STL', 'STT',
       'SWF', 'SYR', 'TPA', 'TUS', 'TVC', 'TYS', 'XNA'], dtype=object)

In [58]:
np.unique(df_4['ORIGIN_AIRPORT_ID'])

array([10140, 10154, 10257, 10397, 10423, 10577, 10581, 10599, 10693,
       10721, 10732, 10785, 10792, 10800, 10821, 10849, 10868, 10990,
       10994, 11042, 11057, 11066, 11193, 11252, 11259, 11278, 11292,
       11298, 11433, 11503, 11537, 11618, 11697, 11986, 11995, 11996,
       12173, 12191, 12197, 12250, 12264, 12265, 12266, 12323, 12339,
       12391, 12397, 12441, 12451, 12478, 12889, 12892, 12945, 12953,
       12954, 13198, 13204, 13232, 13244, 13296, 13303, 13342, 13485,
       13487, 13495, 13502, 13541, 13577, 13796, 13871, 13930, 13931,
       14025, 14027, 14057, 14100, 14107, 14122, 14254, 14262, 14321,
       14492, 14524, 14570, 14576, 14635, 14679, 14683, 14685, 14730,
       14747, 14771, 14831, 14843, 14869, 14893, 14986, 15016, 15024,
       15070, 15096, 15304, 15376, 15380, 15412, 15919])

In [53]:
assert len(np.unique(df_4['ORIGIN_AIRPORT_ID'])) == len(np.unique(df_4['ORIGIN']))

In [55]:
originLen = len(np.unique(df_4['ORIGIN']))
originLen

106

In [56]:
originId_to_origin = dict()

In [67]:
assert compareSimilarCategoricalColumns(col_a = df_4['ORIGIN_AIRPORT_ID'],
    col_b = df_4['ORIGIN'])

In [69]:
df_4.shape

(186308, 47)

In [70]:
df_5 = df_4.drop(labels=['ORIGIN_AIRPORT_ID'], axis=1)
df_5.shape

(186308, 46)

### One-hot encoding for Origin attribute

In [73]:
df_6 = MyOneHotEncoder().encodePandasColAndMerge(data_frame = df_5, col_name = 'ORIGIN')
df_6.shape

(186308, 150)

In [74]:
df_6[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_NUM,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,...,ORIGIN_95,ORIGIN_96,ORIGIN_97,ORIGIN_98,ORIGIN_99,ORIGIN_100,ORIGIN_101,ORIGIN_102,ORIGIN_103,ORIGIN_104
0,3,7,3,7,50,"Honolulu, HI",HI,Hawaii,12478,JFK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,4,2,6,1304,"San Juan, PR",PR,Puerto Rico,12478,JFK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26,6,164,"Sarasota/Bradenton, FL",FL,Florida,12478,JFK,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13,3,429,"New York, NY",NY,New York,14635,RSW,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,18,7,786,"New York, NY",NY,New York,11697,FLL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
